<a href="https://colab.research.google.com/github/Eslam-elnahas/Data_Sets-Project/blob/main/Semi_Sentinel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [12]:
import ee
import geemap # Import the geemap module
# مصادقة وتهيئة Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-eslamelnahas1010')

# تحديد منطقة الاهتمام (AOI) بالقرب من البحر في الدلتا
aoi = ee.Geometry.Rectangle([31.5, 31.5, 31.5, 31.5])  # يمكنك تعديل هذه الإحداثيات حسب المكان المطلوب

# تحديد مجموعة البيانات الخاصة بـ Sentinel-2 Harmonized
dataset = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(aoi) \
    .filterDate('2023-01-01', '2023-12-31') \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()  # اختيار الصورة الأولى ذات أقل نسبة غيوم
# تحويل جميع الحزم إلى UInt16 لضمان التوافق
dataset = dataset.select(['B4', 'B3', 'B2']).uint16()  # استخدام uint16
# تحديد الحزم (RGB) التي نريد استخدامها
visualization = {
    'bands': ['B4', 'B3', 'B2'],  # استخدام الحزم الحمراء، الخضراء، الزرقاء
    'min': 0,
    'max': 3000
}

# عرض الصورة (إذا كنت تستخدم geemap)
Map = geemap.Map() # Create a Map instance
Map.addLayer(dataset, visualization, "Sentinel-2") # Add the layer to the Map instance
Map # Display the Map


# تصدير الصورة كملف GeoTIFF إلى Google Drive
task = ee.batch.Export.image.toDrive(**{
    'image': dataset,
    'description': 'Delta_Image',
    'scale': 10,  # دقة الصورة (10 متر لكل بيكسل)
    'region': aoi,  # المنطقة التي سيتم تصدير الصورة لها
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e9  # زيادة الحد الأقصى للبيكسلات
})

# بدء عملية التصدير
task.start()

print("Image export started. Check your Google Drive for the exported image.")


Image export started. Check your Google Drive for the exported image.


In [13]:
# إنشاء الخريطة وعرض الصورة
Map = geemap.Map(center=[31.25, 30.75], zoom=10)  # إعداد الخريطة مع مركز المنطقة
Map.addLayer(dataset, visualization, "Sentinel-2")  # إضافة الطبقة إلى الخريطة
Map.addLayerControl()  # إضافة تحكم الطبقات في الخريطة
Map  # عرض الخريطة

Map(center=[31.25, 30.75], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [21]:
import rasterio
from PIL import Image
import numpy as np

# تحميل صورة GeoTIFF باستخدام rasterio
tiff_image_path = '/content/drive/MyDrive/Sentinel2_image (1).tif'

with rasterio.open(tiff_image_path) as dataset:
    # قراءة الحزم (bands) كـ numpy array
    img_array = dataset.read([1, 2, 3])  # قراءة الحزم الثلاثة الأولى (RGB)

    # تحويل الصورة من صيغة (Bands, Height, Width) إلى (Height, Width, Bands)
    img_array = np.transpose(img_array, (1, 2, 0))

    # تحويل القيم من uint16 إلى uint8 باستخدام التطبيع (normalization)
    img_array = ((img_array - img_array.min()) / (img_array.max() - img_array.min()) * 255)

    # تحويل نوع البيانات إلى uint8
    img_array = img_array.astype(np.uint8) # This line is crucial for fixing the error

    # تحويل الصورة إلى صورة PIL
    img = Image.fromarray(img_array)

    # حفظ الصورة بصيغة JPEG
    jpg_image_path = 'converted.jpg'
    img.save(jpg_image_path)

    print("تم تحويل الصورة إلى JPEG بنجاح وحفظها في:", jpg_image_path)

تم تحويل الصورة إلى JPEG بنجاح وحفظها في: converted.jpg


In [22]:
from google.colab import files

# تحميل الصورة المحولة إلى جهازك
files.download(jpg_image_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Mounted at /content/drive
